# FuseTS - Peak-Valley detection in S2 NDVI

Welcome to this Jupyter Notebook where we will be working with time series data! Our goal is to download a time series data for a meadow and use the [peak-valley](https://open-eo.github.io/FuseTS/api.html#fusets.peakvalley.peakvalley) detection algorithm from FuseTS to extract insights from the data. Time series analysis is a powerful tool that allows us to understand patterns and trends in data over time, and we will be leveraging this tool to analyze the data for the meadow. With the help of the FuseTS peak-valley detection algorithm, we will be able to identify the highest and lowest points in the data, allowing us to gain a better understanding of the underlying patterns and trends.


**Prerequisites**
* In this notebook we are using openEO to fetch the time series data for the meadow. You can register for a free trial account on the [openEO Platform](https://openeo.cloud/#plans) website.
* jupyter labextension install @jupyter-widgets/jupyterlab-manager, jupyter labextension install @bokeh/jupyter_bokeh@2.0.4

Lets start with importing the different libraries that we need within this notebook.

In [1]:
import openeo
import xarray

### Use `datacube_from_process` to get initial DataCube
Get output datacube from process by passing in the process parameters:

**Mandatory**
- `data`: Initial datacube that contains the data to be processed

**Optional**
- `drop_thr`: Threshold value for the amplitude of the drop in the input feature
- `rec_r`: Threshold value for the amplitude of the recovery, relative to the `drop_delta`
- `slope_thr`: Threshold value for the slope where the peak should start

In [2]:
connection = openeo.connect("openeo.vito.be").authenticate_oidc()
service = 'peakvalley'
namespace = 'u:bramjanssen'

Authenticated using refresh token.


In [3]:
connection.describe_process(service, namespace=namespace)

{'description': '# Peak Valley\n\n## Description\n\n\n',
 'id': 'peakvalley',
 'parameters': [{'description': 'A data cube.',
   'name': 'data',
   'schema': {'subtype': 'raster-cube', 'type': 'object'}},
  {'default': 0.15,
   'description': 'Threshold value for the amplitude of the drop in the input feature',
   'name': 'drop_thr',
   'optional': True,
   'schema': {'type': 'number'}},
  {'default': 1.0,
   'description': 'Threshold value for the amplitude of the recovery, relative to the `drop_delta`',
   'name': 'rec_r',
   'optional': True,
   'schema': {'type': 'number'}},
  {'default': -0.007,
   'description': 'Threshold value for the slope where the peak should start',
   'name': 'slope_thr',
   'optional': True,
   'schema': {'type': 'number'}}],
 'summary': 'Detect peaks and valleys in a time series.'}

Next we define the area of interest, in this case an extent, for which we would like to fetch time series data.

In [4]:
minx, miny, maxx, maxy = (15.179421073198585, 45.80924633589998, 15.185336903822831, 45.81302555710934)
spat_ext = dict(west=minx, east=maxx, north=maxy, south=miny, crs=4326)
temp_ext = ["2021-01-01", "2021-12-31"]

We will create an openEO process to calculate the NDVI time series for our area of interest. We'll begin by using the SENTINEL2_L2A_SENTINELHUB collection, and apply a cloud masking algorithm to remove any interfering clouds before calculating the NDVI values. 

In [18]:
s2 = connection.load_collection('SENTINEL2_L2A_SENTINELHUB',
                                spatial_extent=spat_ext,
                                temporal_extent=temp_ext,
                                bands=["B04","B08","SCL"])
s2 = s2.process("mask_scl_dilation", data=s2, scl_band_name="SCL")
base_ndvi = s2.ndvi(red="B04", nir="B08", target_band='NDVI').band('NDVI')

In [19]:
peakvalley = connection.datacube_from_process(service, namespace=f'https://openeo.vito.be/openeo/1.1/processes/{namespace}/{service}', data=base_ndvi)

/home/bramjanssen/.local/lib/python3.6/site-packages/openeo/metadata.py:255: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")


### Execute job and download result

In [21]:
output_file = './peakvalley.nc'
peakvalley_job = peakvalley.execute_batch(out_format="netcdf", title=f'FuseTS - Peak Valley Detection', job_options={
    'udf-dependency-archives': [
         'https://artifactory.vgt.vito.be:443/auxdata-public/ai4food/fusets_venv.zip#tmp/venv',
        'https://artifactory.vgt.vito.be:443/auxdata-public/ai4food/fusets.zip#tmp/venv_static'
    ]
})
peakvalley_job.get_results().download_file(output_file)

0:00:00 Job 'j-2a11f143e719444f8c3cd85887a14f29': send 'start'
0:00:19 Job 'j-2a11f143e719444f8c3cd85887a14f29': queued (progress N/A)
0:00:24 Job 'j-2a11f143e719444f8c3cd85887a14f29': queued (progress N/A)
0:00:47 Job 'j-2a11f143e719444f8c3cd85887a14f29': queued (progress N/A)
0:00:55 Job 'j-2a11f143e719444f8c3cd85887a14f29': queued (progress N/A)
0:01:05 Job 'j-2a11f143e719444f8c3cd85887a14f29': queued (progress N/A)
0:01:18 Job 'j-2a11f143e719444f8c3cd85887a14f29': queued (progress N/A)
0:01:33 Job 'j-2a11f143e719444f8c3cd85887a14f29': queued (progress N/A)
0:01:53 Job 'j-2a11f143e719444f8c3cd85887a14f29': running (progress N/A)
0:02:22 Job 'j-2a11f143e719444f8c3cd85887a14f29': running (progress N/A)
0:02:52 Job 'j-2a11f143e719444f8c3cd85887a14f29': running (progress N/A)
0:03:30 Job 'j-2a11f143e719444f8c3cd85887a14f29': running (progress N/A)
0:04:17 Job 'j-2a11f143e719444f8c3cd85887a14f29': running (progress N/A)
0:05:16 Job 'j-2a11f143e719444f8c3cd85887a14f29': running (progress 

PosixPath('peakvalley.nc')

Now that we have calculated the NDVI time series, we can utilize it to execute the peak valley algorithm that is part of the FuseTS algorithm. The peak valley algorithm is a powerful tool that allows us to detect significant changes in the vegetation patterns over time. 

## Explore the results

In [22]:
ds = xarray.load_dataset(output_file)

In [24]:
import holoviews as hv


hv.extension('bokeh')
%output holomap='scrubber'
%opts Image style(cmap='viridis') plot[colorbar=True]
%opts Image [width=500, height=400]
hv_ds = hv.Dataset(ds)
hv_ds.to(hv.Image, ['x', 'y'])

ModuleNotFoundError: No module named 'holoviews'